# Lab 3: Policy Search

## Task

Write agents able to play [*Nim*](https://en.wikipedia.org/wiki/Nim), with an arbitrary number of rows and an upper bound $k$ on the number of objects that can be removed in a turn (a.k.a., *subtraction game*).

The player **taking the last object wins**.

* Task3.1: An agent using fixed rules based on *nim-sum* (i.e., an *expert system*)
* Task3.2: An agent using evolved rules
* Task3.3: An agent using minmax
* Task3.4: An agent using reinforcement learning

## Deadlines ([AoE](https://en.wikipedia.org/wiki/Anywhere_on_Earth))

* Sunday, December 4th for Task3.1 and Task3.2
* Sunday, December 11th for Task3.3 and Task3.4
* Sunday, December 18th for all reviews

In [1]:
import logging
from collections import namedtuple
import random
from typing import Callable
from copy import deepcopy
from functools import reduce
from operator import xor

logging.basicConfig(format="%(asctime)s %(levelname)s: %(message)s", level=logging.DEBUG)

## The *Nim* and *Nimply* classes

In [2]:
Nimply = namedtuple("Nimply", "row, num_objects")
Genome = namedtuple("Genome", "aggressivity, longest_first, how_many")
Individual = namedtuple("Individual", "genome, fitness")

In [3]:
class Nim:
    def __init__(self, num_rows: int, k: int = None) -> None:
        self._rows = [i * 2 + 1 for i in range(num_rows)]
        self._k = k

    def __bool__(self):
        return sum(self._rows) > 0

    def __str__(self):
        return "<" + " ".join(str(_) for _ in self._rows) + ">"

    @property
    def rows(self) -> tuple:
        return tuple(self._rows)

    @property
    def k(self) -> int:
        return self._k

    def nimming(self, ply: Nimply) -> None:
        row, num_objects = ply     
        assert self._rows[row] >= num_objects
        assert self._k is None or num_objects <= self._k
        self._rows[row] -= num_objects

## Task 3.1

In [4]:
def pure_random(state: Nim) -> Nimply:
    row = random.choice([r for r, c in enumerate(state.rows) if c > 0])
    num_objects = random.randint(1, state.rows[row])
    return Nimply(row, num_objects)

In [5]:
def gabriele(state: Nim) -> Nimply:
    """Pick always the maximum possible number of the lowest row"""
    possible_moves = [(r, o) for r, c in enumerate(state.rows) for o in range(1, c + 1)]
    return Nimply(*max(possible_moves, key=lambda m: (-m[0], m[1])))

In [6]:
def nim_sum(state: Nim) -> int:
    result = reduce(xor, state.rows)
    return result


def cook_status(state: Nim) -> dict:
    cooked = dict()
    cooked["possible_moves"] = [
        (r, o) for r, c in enumerate(state.rows) for o in range(1, c + 1) if state.k is None or o <= state.k
    ]
    # cooked["active_rows_number"] = sum(o > 0 for o in state.rows)
    # cooked["shortest_row"] = min((x for x in enumerate(state.rows) if x[1] > 0), key=lambda y: y[1])[0]
    # cooked["longest_row"] = max((x for x in enumerate(state.rows)), key=lambda y: y[1])[0]
    cooked["nim_sum"] = nim_sum(state)

    brute_force = list()
    for m in cooked["possible_moves"]:
        tmp = deepcopy(state)
        tmp.nimming(m)
        brute_force.append((m, nim_sum(tmp)))
    cooked["brute_force"] = brute_force

    return cooked

In [7]:
def optimal_strategy(state: Nim) -> Nimply:
    data = cook_status(state)
    return next((bf for bf in data["brute_force"] if bf[1] == 0), random.choice(data["brute_force"]))[0]

## Task 3.2

In [8]:
def make_strategy(genome: dict) -> Callable:
    def evolvable(state: Nim) -> Nimply:
        data = cook_status(state)

        if random.random() < genome["p"]:
            ply = Nimply(data["shortest_row"], random.randint(1, state.rows[data["shortest_row"]]))
        else:
            ply = Nimply(data["longest_row"], random.randint(1, state.rows[data["longest_row"]]))

        return ply

    return evolvable

def make_strategy_g(genome: Genome) -> Callable:
    def evolvable(state: Nim) -> Nimply:
        # data = cook_status(state)
        aggressive: bool = random.random() < genome.aggressivity
        longest_first: bool = random.random() < genome.longest_first
        how_many_coeff: float = genome.how_many

        # Sort the rows based on the number of elements, the sort is descending or ascending based on if longest_first or not
        row_indexes = sorted((i for i in range(len(state.rows)) if state.rows[i] > 0), key=lambda elem: state.rows[elem], reverse=longest_first)

        # Select randomly one of the first 50% rows
        selected_row_index = random.choice(row_indexes[:int(0.5*len(row_indexes))+1])

        # Decide to take or nt at least 1 or half of the objects
        take_at_least = 0 if not aggressive else state.rows[selected_row_index]//2
        
        # Decide to take or not a part of the remaining objects
        take_n = max(1, min(take_at_least + int(state.rows[selected_row_index]//2*how_many_coeff), state.rows[selected_row_index]))
        ply = Nimply(selected_row_index, take_n)

        return ply

    return evolvable

In [9]:
NIM_SIZE = 10


def evaluate(strategy: Callable, opponent: Callable=optimal_strategy, NUM_MATCHES=100) -> float:
    strategies = (strategy, opponent)
    won = 0

    for m in range(NUM_MATCHES):
        nim = Nim(NIM_SIZE)
        player = 0
        while nim:
            ply = strategies[player](nim)
            nim.nimming(ply)
            player = 1 - player
        if player == 1:
            won += 1
    return won / NUM_MATCHES

## Evolve genome

In [10]:
def tournament(population, k=10):
    return max(random.choices(population, k=k), key=lambda x: x.fitness)

def make_offspring(p1: Individual, p2: Individual, current_best_strategy):
    aggressivity = p1.genome.aggressivity if random.random() > 0.5 else p2.genome.aggressivity
    aggressivity += random.gauss(0, 0.25)
    aggressivity = max(0, min(aggressivity, 1))

    longest_first = p1.genome.longest_first if random.random() > 0.5 else p2.genome.longest_first
    longest_first += random.gauss(0, 0.25)
    longest_first = max(0, min(longest_first, 1))
    
    how_many = p1.genome.how_many if random.random() > 0.5 else p2.genome.how_many
    how_many += random.gauss(0, 0.25)
    how_many = max(0, min(how_many, 1))

    new_genome = Genome(aggressivity, longest_first, how_many)
    individual = Individual(new_genome, evaluate(make_strategy_g(new_genome), current_best_strategy))
    
    return individual

In [11]:
ITERATIONS = 20
POPULATION_SIZE = 40
OFFSPRING_SIZE = 200
genomes = [Genome(0.5 + random.random()/10, 0.5 + random.random()/10, 0.5 + random.random()/10) for _ in range(POPULATION_SIZE)]

population = list(map(lambda genome: Individual(genome, evaluate(make_strategy_g(genome), gabriele)), genomes))
for i in range(ITERATIONS):
    logging.debug(f"Starting iteration {i}. Current fitness: {population[0].fitness}")
    logging.debug(f"Current best:\nAggressivity\tLongest first\tHow many\n\
{population[0].genome.aggressivity:.2f}\t\t{population[0].genome.longest_first:.2f}\t\t{population[0].genome.how_many:.2f}")
    offspring = []
    for _ in range(OFFSPRING_SIZE):
        p1, p2 = tournament(population, k=POPULATION_SIZE), tournament(population, k=POPULATION_SIZE)
        o = make_offspring(p1, p2, make_strategy_g(population[0].genome))
        offspring.append(o)
    population.extend(offspring)
    population = sorted(population, key=lambda individual: individual.fitness, reverse=True)[:POPULATION_SIZE]

evolved_individual = population[0]

2022-12-02 18:01:59,481 DEBUG: Starting iteration 0. Current fitness: 0.1
2022-12-02 18:01:59,482 DEBUG: Current best:
Aggressivity	Longest first	How many
0.57		0.57		0.57
2022-12-02 18:02:04,768 DEBUG: Starting iteration 1. Current fitness: 0.69
2022-12-02 18:02:04,768 DEBUG: Current best:
Aggressivity	Longest first	How many
0.96		0.11		0.47
2022-12-02 18:02:09,874 DEBUG: Starting iteration 2. Current fitness: 0.69
2022-12-02 18:02:09,875 DEBUG: Current best:
Aggressivity	Longest first	How many
0.96		0.11		0.47
2022-12-02 18:02:14,711 DEBUG: Starting iteration 3. Current fitness: 0.77
2022-12-02 18:02:14,712 DEBUG: Current best:
Aggressivity	Longest first	How many
0.95		0.00		0.57
2022-12-02 18:02:18,840 DEBUG: Starting iteration 4. Current fitness: 0.77
2022-12-02 18:02:18,840 DEBUG: Current best:
Aggressivity	Longest first	How many
0.95		0.00		0.57
2022-12-02 18:02:23,041 DEBUG: Starting iteration 5. Current fitness: 0.96
2022-12-02 18:02:23,042 DEBUG: Current best:
Aggressivity	Lon

## Final match

In [12]:
evaluate(make_strategy_g(evolved_individual.genome), optimal_strategy, NUM_MATCHES=1000)

0.0

In [13]:
strategies = (make_strategy_g(evolved_individual.genome), optimal_strategy)

nim = Nim(11)
logging.debug(f"status: Initial board  -> {nim}")
player = 0
while nim:
    ply = strategies[player](nim)
    nim.nimming(ply)
    logging.debug(f"status: After player {player} -> {nim}")
    player = 1 - player
winner = 1 - player
logging.info(f"status: Player {winner} won!")

2022-12-02 18:03:34,895 DEBUG: status: Initial board  -> <1 3 5 7 9 11 13 15 17 19 21>
2022-12-02 18:03:34,897 DEBUG: status: After player 0 -> <1 3 5 7 9 11 13 15 17 19 6>
2022-12-02 18:03:34,899 DEBUG: status: After player 1 -> <1 3 1 7 9 11 13 15 17 19 6>
2022-12-02 18:03:34,901 DEBUG: status: After player 0 -> <1 3 1 7 9 11 13 15 17 19 2>
2022-12-02 18:03:34,902 DEBUG: status: After player 1 -> <1 3 1 3 9 11 13 15 17 19 2>
2022-12-02 18:03:34,903 DEBUG: status: After player 0 -> <1 3 1 3 9 11 13 5 17 19 2>
2022-12-02 18:03:34,906 DEBUG: status: After player 1 -> <1 3 1 3 3 11 13 5 17 19 2>
2022-12-02 18:03:34,907 DEBUG: status: After player 0 -> <1 3 1 3 3 11 13 5 17 6 2>
2022-12-02 18:03:34,908 DEBUG: status: After player 1 -> <1 3 1 3 3 11 13 5 4 6 2>
2022-12-02 18:03:34,909 DEBUG: status: After player 0 -> <1 3 1 3 3 11 13 2 4 6 2>
2022-12-02 18:03:34,910 DEBUG: status: After player 1 -> <1 3 1 3 3 11 10 2 4 6 2>
2022-12-02 18:03:34,911 DEBUG: status: After player 0 -> <1 2 1 3 